# Initial Setup and Config

In [1]:
import data_functions
import os
from IPython.display import display
import json
import importlib


# --- Configuration: Define Project Paths ---
# You can change BASE_PROJECT_DIR if your data/results aren't relative to the notebook
BASE_PROJECT_DIR = '.' # Assumes seqs, summary, results are subdirs of the notebook's dir or a linked dir

# Define specific directories relative to the base
SEQS_DIR = os.path.join(BASE_PROJECT_DIR, 'seqs')
SUMMARY_DIR = os.path.join(BASE_PROJECT_DIR, 'summary')
RESULTS_DIR = os.path.join(BASE_PROJECT_DIR, 'results')

# Create results directory if it doesn't exist
os.makedirs(RESULTS_DIR, exist_ok=True)

print(f"Using Sequences Directory: {os.path.abspath(SEQS_DIR)}")
print(f"Using Summary Directory:   {os.path.abspath(SUMMARY_DIR)}")
print(f"Using Results Directory:   {os.path.abspath(RESULTS_DIR)}")


Using Sequences Directory: c:\gitsync\nanopore-consensus-benchmark\seqs
Using Summary Directory:   c:\gitsync\nanopore-consensus-benchmark\summary
Using Results Directory:   c:\gitsync\nanopore-consensus-benchmark\results


## Load Runs

In [2]:
runs_df, runs_dict = data_functions.discover_runs(SEQS_DIR)

print("Discovered Runs:")
display(runs_df)
print("\nRuns Dictionary:")
print(runs_dict)

Discovered Runs:


,dorado,guppy,Both Available
Run ID,,,
OMDL1,True,True,True
OMDL2,True,True,True
OMDL3,True,True,True
OMDL4,True,True,True
OMDL5,True,True,True
OMDL6,True,True,True
OMDL7,True,True,True
OMDL8,True,True,True
OMDL9,True,True,True



Runs Dictionary:
{'OMDL10': {'dorado': True, 'guppy': True}, 'OMDL12': {'dorado': True, 'guppy': True}, 'OMDL13': {'dorado': True, 'guppy': True}, 'OMDL1': {'dorado': True, 'guppy': True}, 'OMDL2': {'dorado': True, 'guppy': True}, 'OMDL3': {'dorado': True, 'guppy': True}, 'OMDL4': {'dorado': True, 'guppy': True}, 'OMDL5': {'dorado': True, 'guppy': True}, 'OMDL6': {'dorado': True, 'guppy': True}, 'OMDL7': {'dorado': True, 'guppy': True}, 'OMDL8': {'dorado': True, 'guppy': True}, 'OMDL9': {'dorado': True, 'guppy': True}}


## Test loading sequences

In [3]:
importlib.reload(data_functions)
test_run_id = 'OMDL1' # Replace with a valid run ID from your data
test_basecaller = 'dorado' # Replace 'dorado' or 'guppy' as available

loaded_sequences = data_functions.load_sequences(test_run_id, test_basecaller, SEQS_DIR)

if loaded_sequences is not None:
    print(f"Successfully loaded data for {test_run_id} {test_basecaller}.")
    print(f"Found data for {len(loaded_sequences)} unique sample IDs.")

    # Example: Inspect data for one sample ID (replace 'OMDLxxxxx' with a real ID)
    example_sample_id = list(loaded_sequences.keys())[0] # Get the first sample ID found
    print(f"\nData for sample ID '{example_sample_id}':")
    # Pretty print the list of sequence dictionaries for this sample
    print(json.dumps(loaded_sequences[example_sample_id], indent=2, default=str)) # Use default=str to handle SeqRecord object if present

    # Verify structure of one sequence entry
    first_seq_data = loaded_sequences[example_sample_id][0]
    print("\nStructure of one sequence entry:")
    print(f"  Header: {first_seq_data.get('header')[:50]}...") # Show first 50 chars
    print(f"  Length: {first_seq_data.get('length')}")
    print(f"  RiC: {first_seq_data.get('ric')}")
    print(f"  Sequence snippet: {first_seq_data.get('sequence')[:50]}...") # Show first 50 chars
else:
    print(f"Failed to load data for {test_run_id} {test_basecaller}. Check file path and format.")

Successfully loaded data for OMDL1 dorado.
Found data for 152 unique sample IDs.

Data for sample ID 'OMDL00009':
[
  {
    "header": "ONT01.09-A02-OMDL00009-iNat169115711-1 ric=388",
    "sequence": "GTAAAAGTCGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTATTGAATGAAATGTGCTTAGACTGATGCTGGCTCTATGGAGCATGGTGCACGTCTAGCATTCATCTCTGTCACCTGTGCACCATTTGTAGGCCTGGTTTGGGGATTTGCAATGCAGCTTTCCTTATTGGCATTTATTCAGGCCTATGTAATAATATTGTACAACACTGTATATATATCAAGTTGAGAATAATGTTATTGTAACAAACCTTATACAACTTTCAACAACGGATCTCTTGGCTCTCGCATCGATGAAGAACGCAGCGAAATGCGATAAGTAATGTGAATTGCAGAATTCAGTGAATCATCGAATCTTTGAACGCACCTTGCGCTCCTTGGTATTCCGAAGAGCATGCCTGTTTGAGTGTCATTAAATTCTCAACCTTTCTTGGCTTTATAGCTAAGTTTAAGGCTTGGTTGTGGAGGCTGCGGGCTTCTTAGAAGTCGGCTCTTCTTAAATATATTAGCGAAACCTTTTTGCTGATCATCTCTGGTGTGATAGTTTATCTACACCATTAAGAACAGCTTTGTGTGGTTTTAGCTTCTAACTGTCTCTTGGACAATTTATTGACAATCTGACCTCAAATCAGGTAGGATTACCCGCTGAACTTAAGATAA",
    "length": 665,
    "ric": 388,
    "seq_object": "ID: ONT01.09-A02-OMDL00009-iNat169115711-1\nName: ONT01.09-A02-OMDL00009-iNat

## Test K-mer matching

In [ ]:
importlib.reload(data_functions)
seq_a = "ATGCGATGCGATGCG"
seq_b = "ATGCGATGCGATGCG" # Identical
seq_c = "ATGCGATTCGATGCG" # One mismatch
seq_d = "AAAAAAAAAAAAAAA" # Different
seq_e = "ATGCG"             # Too short for k=7
seq_f = ""                # Empty

k_val = 7
print(f"Similarity A vs B (k={k_val}): {data_functions.calculate_kmer_similarity(seq_a, seq_b, k=k_val):.2f}%")
print(f"Similarity A vs C (k={k_val}): {data_functions.calculate_kmer_similarity(seq_a, seq_c, k=k_val):.2f}%") # Test mismatch 22.22%
print(f"Similarity B vs A (k={k_val}): {data_functions.calculate_kmer_similarity(seq_b, seq_a, k=k_val):.2f}%") # Should be symmetric? Test.
print(f"Similarity A vs D (k={k_val}): {data_functions.calculate_kmer_similarity(seq_a, seq_d, k=k_val):.2f}%") # Test different sequence 0.00%
print(f"Similarity A vs E (k={k_val}): {data_functions.calculate_kmer_similarity(seq_a, seq_e, k=k_val):.2f}%") # Test too short sequence 0.00%
print(f"Similarity A vs F (k={k_val}): {data_functions.calculate_kmer_similarity(seq_a, seq_f, k=k_val):.2f}%") # Test empty sequence 0.00%

k_val = 3
print(f"\nSimilarity A vs C (k={k_val}): {data_functions.calculate_kmer_similarity(seq_a, seq_c, k=k_val):.2f}%") # Test smaller k, 76.92%
print(f"Similarity A vs E (k={k_val}): {data_functions.calculate_kmer_similarity(seq_a, seq_e, k=k_val):.2f}%") # Should work now

## Test Global Alignments

In [ ]:
importlib.reload(data_functions)
seq_a = "ATGCGATGCGATGCG"
seq_b = "ATGCGATGCGATGCG" # Identical
seq_c = "ATGCGATTCGATGCG" # One mismatch
seq_d = "AAAAAAAAAAAAAAA" # Different
seq_indel = "ATGCGATG---ATGCG" # Example with deletion relative to A

align_ab = data_functions.align_sequences(seq_a, seq_b)
align_ac = data_functions.align_sequences(seq_a, seq_c)
align_ad = data_functions.align_sequences(seq_a, seq_d)
align_a_indel = data_functions.align_sequences(seq_a, seq_indel)

print("Alignment A vs B:")
print(json.dumps(align_ab, indent=2, default=str)) # Use default=str to handle alignment obj if needed

print("\nAlignment A vs C:")
print(json.dumps(align_ac, indent=2, default=str))

print("\nAlignment A vs D:")
print(json.dumps(align_ad, indent=2, default=str))

print("\nAlignment A vs Indel:")
print(json.dumps(align_a_indel, indent=2, default=str))

# Test empty sequence
align_a_empty = data_functions.align_sequences(seq_a, "")
print("\nAlignment A vs Empty:")
print(align_a_empty)

## Sequence Matching Logic

In [6]:
importlib.reload(data_functions)
# --- Define Test Sequences ---
seq_a = "ATGCGATGCGATGCG"     # Base sequence
seq_b = "ATGCGATGCGATGCG"     # Identical to A
seq_c = "ATGCGATTCGATGCG"     # One mismatch vs A
seq_d = "AAAAAAAAAAAAAAA"     # Very different from A
seq_indel = "ATGCGATG---ATGCG"  # Contains gaps (Note: align_sequences takes raw seqs, not pre-aligned)
seq_e = "ATGCG"                 # Short sequence
seq_f = ""                    # Empty sequence
# Create sequences similar to D and C for testing many:many and ambiguous cases
seq_d_like = "AAAAAAAAAAAAAAC" # Similar to D
seq_c_prime = "ATGCGATTCAATGCG" # Similar to C (two mismatches vs A)

# --- Helper Function to Create Sequence Records ---
# Mimics the structure produced by load_sequences
def create_record(seq_id: str, sequence: str, ric: int, source: str, sample: str, rep_num: int = 1):
    """Creates a dictionary representing a sequence record."""
    # Create a somewhat realistic header based on inputs
    header = f">ONT01.01-{sample}-{seq_id}-iNat0000{rep_num} ric={ric}"
    return {
        'header': header,
        'sequence': sequence,
        'length': len(sequence),
        'ric': ric,
        'seq_object': None # Placeholder, not needed for matching logic testing
    }

# --- Create Mock Dorado Sequences Dictionary ---
mock_dorado_seqs = {
    # Scenario S1: Simple 1:1 High Identity
    'S1': [create_record('D_S1_1', seq_a, 100, 'dorado', 'S1')],
    # Scenario S2: Simple 1:1 Lower Identity
    'S2': [create_record('D_S2_1', seq_a, 90, 'dorado', 'S2')],
    # Scenario S3: Unmatched Pair
    'S3': [create_record('D_S3_1', seq_a, 80, 'dorado', 'S3')],
    # Scenario S4: Dorado Only Sample
    'S4': [create_record('D_S4_1', seq_a, 70, 'dorado', 'S4')],
    # Scenario S6: 1 Dorado, 2 Guppy
    'S6': [create_record('D_S6_1', seq_a, 110, 'dorado', 'S6')],
    # Scenario S7: 2 Dorado, 2 Guppy (Clear matches)
    'S7': [
        create_record('D_S7_1', seq_a, 120, 'dorado', 'S7', rep_num=1),
        create_record('D_S7_2', seq_d, 50, 'dorado', 'S7', rep_num=2)
    ],
    # Scenario S8: 1 Dorado, 2 Guppy (Ambiguous matches)
    'S8': [create_record('D_S8_1', seq_a, 130, 'dorado', 'S8')],
}

# --- Create Mock Guppy Sequences Dictionary ---
mock_guppy_seqs = {
    # Scenario S1: Simple 1:1 High Identity
    'S1': [create_record('G_S1_1', seq_b, 95, 'guppy', 'S1')],
    # Scenario S2: Simple 1:1 Lower Identity
    'S2': [create_record('G_S2_1', seq_c, 85, 'guppy', 'S2')],
    # Scenario S3: Unmatched Pair
    'S3': [create_record('G_S3_1', seq_d, 75, 'guppy', 'S3')],
    # Scenario S5: Guppy Only Sample
    'S5': [create_record('G_S5_1', seq_a, 65, 'guppy', 'S5')],
    # Scenario S6: 1 Dorado, 2 Guppy
    'S6': [
        create_record('G_S6_1', seq_b, 105, 'guppy', 'S6', rep_num=1), # Should match D_S6_1 well
        create_record('G_S6_2', seq_c, 45, 'guppy', 'S6', rep_num=2)  # Should match D_S6_1 less well
    ],
    # Scenario S7: 2 Dorado, 2 Guppy (Clear matches)
    'S7': [
        create_record('G_S7_1', seq_b, 115, 'guppy', 'S7', rep_num=1), # Should match D_S7_1 (A)
        create_record('G_S7_2', seq_d_like, 55, 'guppy', 'S7', rep_num=2) # Should match D_S7_2 (D)
    ],
    # Scenario S8: 1 Dorado, 2 Guppy (Ambiguous matches)
    'S8': [
        create_record('G_S8_1', seq_c, 125, 'guppy', 'S8', rep_num=1),       # Similar match to D_S8_1 (A)
        create_record('G_S8_2', seq_c_prime, 110, 'guppy', 'S8', rep_num=2) # Also similar match to D_S8_1 (A)
    ],
}

print("Mock data dictionaries created: mock_dorado_seqs, mock_guppy_seqs")
# Print a sample entry to verify structure
example_sample_id = 'S7'
print(f"\nExample entry for {example_sample_id} in mock_dorado_seqs:")
print(json.dumps(mock_dorado_seqs.get(example_sample_id, 'Not Found'), indent=2))
print(f"\nExample entry for {example_sample_id} in mock_guppy_seqs:")
print(json.dumps(mock_guppy_seqs.get(example_sample_id, 'Not Found'), indent=2))

matched, dorado_unmatched, guppy_unmatched = data_functions.match_sequences(mock_dorado_seqs, mock_guppy_seqs)

print(f"Matched pairs: {len(matched)}")
print(f"Dorado-only: {len(dorado_unmatched)}")
print(f"Guppy-only: {len(guppy_unmatched)}")

print("\n--- Matched Pairs ---")
for pair in matched:
    print(f"  Sample: {pair['sample_id']}, "
          f"D_Header: {pair['dorado'].get('header','N/A')}, "
          f"G_Header: {pair['guppy'].get('header','N/A')}, "
          f"Identity: {pair['alignment']['identity']:.2f}%, "
          f"Multiple: {pair['multiple_matches']}, "
          f"Confidence: {pair['match_confidence']}")

print("\n--- Dorado Only ---")
for item in dorado_unmatched:
    print(f"  Sample: {item['sample_id']}, Header: {item['record'].get('header','N/A')}")

print("\n--- Guppy Only ---")
for item in guppy_unmatched:
    print(f"  Sample: {item['sample_id']}, Header: {item['record'].get('header','N/A')}")

Mock data dictionaries created: mock_dorado_seqs, mock_guppy_seqs

Example entry for S7 in mock_dorado_seqs:
[
  {
    "header": ">ONT01.01-S7-D_S7_1-iNat00001 ric=120",
    "sequence": "ATGCGATGCGATGCG",
    "length": 15,
    "ric": 120,
    "seq_object": null
  },
  {
    "header": ">ONT01.01-S7-D_S7_2-iNat00002 ric=50",
    "sequence": "AAAAAAAAAAAAAAA",
    "length": 15,
    "ric": 50,
    "seq_object": null
  }
]

Example entry for S7 in mock_guppy_seqs:
[
  {
    "header": ">ONT01.01-S7-G_S7_1-iNat00001 ric=115",
    "sequence": "ATGCGATGCGATGCG",
    "length": 15,
    "ric": 115,
    "seq_object": null
  },
  {
    "header": ">ONT01.01-S7-G_S7_2-iNat00002 ric=55",
    "sequence": "AAAAAAAAAAAAAAC",
    "length": 15,
    "ric": 55,
    "seq_object": null
  }
]
Processing 6 samples common to both Dorado and Guppy...
Processing 1 samples unique to Dorado...
Processing 1 samples unique to Guppy...
Matching complete. Found 6 matched pairs, 2 Dorado-only sequences, 4 Guppy-only sequen